In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.optimize import minimize

In [2]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [3]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
1,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
2,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
3,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
4,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
5,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
6,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
7,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
8,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
9,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [4]:
#Scale for better intepretation

data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [5]:
# Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)

In [6]:
data = data.sort_values(['Year', 'ID']).reset_index(drop=True)

# Outside share

In [7]:
data.loc[data['Year'] == 2013, 'Market_share'] = data.loc[data['Year'] == 2013, 'Sales'] / 180632
data.loc[data['Year'] == 2014, 'Market_share'] = data.loc[data['Year'] == 2014, 'Sales'] / 188406
data.loc[data['Year'] == 2015, 'Market_share'] = data.loc[data['Year'] == 2015, 'Sales'] / 206653
data.loc[data['Year'] == 2016, 'Market_share'] = data.loc[data['Year'] == 2016, 'Sales'] / 222471
data.loc[data['Year'] == 2017, 'Market_share'] = data.loc[data['Year'] == 2017, 'Sales'] / 221471
data.loc[data['Year'] == 2018, 'Market_share'] = data.loc[data['Year'] == 2018, 'Sales'] / 252328
data.loc[data['Year'] == 2019, 'Market_share'] = data.loc[data['Year'] == 2019, 'Sales'] / 258727
data.loc[data['Year'] == 2020, 'Market_share'] = data.loc[data['Year'] == 2020, 'Sales'] / 230060
data.loc[data['Year'] == 2021, 'Market_share'] = data.loc[data['Year'] == 2021, 'Sales'] / 222210
data.loc[data['Year'] == 2022, 'Market_share'] = data.loc[data['Year'] == 2022, 'Sales'] / 181030
data.loc[data['Year'] == 2023, 'Market_share'] = data.loc[data['Year'] == 2023, 'Sales'] / 203690

In [8]:
data['outside_share'] = 1 - data.groupby('Year')['Market_share'].transform('sum')
data[['Market_share', 'outside_share']].describe()

,Market_share,outside_share
count,334.000000,334.000000
mean,0.002128,0.840911
std,0.005618,0.108763
min,0.000004,0.696180
25%,0.000123,0.696180
50%,0.000715,0.830851
75%,0.002161,0.938460
max,0.088139,0.997525


# IV

In [9]:
fun.BLP(data, 'Range')
fun.BLP(data, 'HP')
fun.BLP(data, 'Chargetime')

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,outside_share,Range_BLP,HP_BLP,Chargetime_BLP
0,10,2013,0.000006,BMW,I3,29.5,25.00000,16.7,18,Hatchback,B,DE,1,1.0,0,0.997525,168.5,111.7,237
1,132,2013,0.001168,Nissan,Leaf,32.8,25.36900,14.7,43,Hatchback,C,JP,211,1.0,0,0.997525,165.2,113.7,212
2,158,2013,0.000509,Renault,Zoe,36.5,17.31500,13.4,56,Hatchback,B,FR,92,1.0,0,0.997525,161.5,115.0,199
3,167,2013,0.000006,Smart,Fortwo,12.7,21.04575,8.0,60,Hatchback,A,DE,1,1.0,0,0.997525,185.3,120.4,195
4,173,2013,0.000620,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,F,US,112,1.0,0,0.997525,137.1,60.9,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,187,2023,0.000388,Volkswagen,up!,25.6,17.48530,8.1,48,Hatchback,A,DE,79,1.0,0,0.696180,4191.8,2652.3,3227
330,188,2023,0.002538,Volvo,C40,46.6,43.06649,40.2,28,SUV,C,SE,517,1.0,0,0.696180,4170.8,2620.2,3247
331,189,2023,0.000020,Volvo,EX30,47.5,36.82450,26.8,28,SUV,B,SE,4,1.0,0,0.696180,4169.9,2633.6,3247
332,190,2023,0.009647,Volvo,XC40,45.7,43.92666,40.2,28,SUV,C,SE,1965,1.0,0,0.696180,4171.7,2620.2,3247


# Pure Logit

In [10]:
formula = 'np.log(Market_share/outside_share) ~ 1 + [Price ~ Range_BLP + HP_BLP + Chargetime_BLP] + Range + HP + Chargetime + China '
IV = IV2SLS.from_formula(formula, data).fit(cov_type='robust')
IV.first_stage

,Price
R-squared,0.5822
Partial R-squared,0.0261
Shea's R-squared,0.0261
Partial F-statistic,17.145
P-value (Partial F-stat),0.0007
Partial F-stat Distn,chi2(3)
==========================,===========
Intercept,-1.3281
,(-0.1735)
Range,0.1875


In [11]:
IV.summary#.tables[1]

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                                   IV-2SLS Estimation Summary                                   
================================================================================================
Dep. Variable:     np.log(Market_share / outside_share)   R-squared:                      0.1614
Estimator:                                      IV-2SLS   Adj. R-squared:                 0.1486
No. Observations:                                   334   F-statistic:                    101.16
Date:                                  Tue, Jun 04 2024   P-value (F-stat)                0.0000
Time:                                          18:29:37   Distribution:                  chi2(5)
Cov. Estimator:                                  robust                                         
                                                                                                
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -9.0442     0.7141    -12.665     0.0000     -10.444     -7.6446
Range          0.0979     0.0207     4.7296     0.0000      0.0573      0.1384
HP            -0.0687     0.0370    -1.8581     0.0632     -0.1412      0.0038
Chargetime    -0.0313     0.0117    -2.6686     0.0076     -0.0543     -0.0083
China         -0.5112     0.4488    -1.1391     0.2547     -1.3908      0.3684
Price          0.0121     0.0324     0.3734     0.7089     -0.0514      0.0757
==============================================================================

Endogenous: Price
Instruments: Range_BLP, HP_BLP, Chargetime_BLP
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Willingness to pay

In [12]:
alpha = IV.params[-1] # Price coefficient
beta = IV.params[:-1]
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(IV.params.index[1:-1])): #[1:-1] to exclude constant and price
    print('W2P:', IV.params.index[1:-1][i], beta_alpha_ratio[i])

W2P: Range -8.085366691412226
W2P: HP 5.676931855016096
W2P: Chargetime 2.583914604479259
W2P: China 42.22586896075744


NOTE: English delimiter\
6,192 DKK for every 10 increase in Range\
10,150 DKK for every 10 increase in HP\
-2,632 DKK for every 10 increase in Chargetime\
-169,891 DKK for Chinese cars OR -16,989 DKK for Chinese cars???

# Substitution patterns

In [13]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
X = logit_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = logit_data['Price']
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [14]:
probability_ratio = fun.probability_ratio(logit_data, 2013)
probability_ratio

Model,I3,Leaf,Zoe,Fortwo,Model S,up!
Model,,,,,,
I3,1.0,1.373144,1.447378,11.114201,1.251912,2.273776
Leaf,0.728256,1.0,1.054061,8.093978,0.911712,1.65589
Zoe,0.690904,0.948712,1.0,7.67885,0.864951,1.570962
Fortwo,0.089975,0.123549,0.130228,1.0,0.112641,0.204583
Model S,0.798778,1.096838,1.156134,8.877782,1.0,1.816243
up!,0.439797,0.603905,0.636553,4.887992,0.550587,1.0


In [15]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
marginal_effects = fun.marginal_effects(logit_data, IV)
marginal_effects

,Intercept,Range,HP,Chargetime,China,Price
Model_year,,,,,,
I3_2013,-1.769326,0.019149,-0.013445,-0.00612,-0.100006,0.002368
Leaf_2013,-1.415955,0.015325,-0.01076,-0.004897,-0.080033,0.001895
Zoe_2013,-1.35995,0.014719,-0.010334,-0.004704,-0.076868,0.00182
Fortwo_2013,-0.21192,0.002294,-0.00161,-0.000733,-0.011978,0.000284
Model S_2013,-1.516799,0.016416,-0.011526,-0.005246,-0.085733,0.00203
...,...,...,...,...,...,...
up!_2023,-0.015707,0.00017,-0.000119,-0.000054,-0.000888,0.000021
C40_2023,-0.034357,0.000372,-0.000261,-0.000119,-0.001942,0.000046
EX30_2023,-0.086863,0.00094,-0.00066,-0.0003,-0.00491,0.000116


In [16]:
marginal_effects[IV.params.index].mean()

Intercept    -0.261736
Range         0.002833
HP           -0.001989
Chargetime   -0.000905
China        -0.014794
Price         0.000350
dtype: float64

In [17]:
elasticity = fun.elasticity(logit_data, IV)
elasticity

,Intercept,Range,HP,Chargetime,China,Price
Model_year,,,,,,
I3_2013,-6.630923,2.11708,-0.841484,-0.412825,-0.0,0.221899
Leaf_2013,-7.286711,2.586704,-0.813962,-1.083727,-0.0,0.247444
Zoe_2013,-7.376849,2.914104,-0.751157,-1.428824,-0.0,0.170976
Fortwo_2013,-8.827044,1.213278,-0.536612,-1.831835,-0.0,0.248669
Model S_2013,-7.116522,4.690579,-3.650286,-0.738429,-0.0,0.685454
...,...,...,...,...,...,...
up!_2023,-9.028442,2.501463,-0.555716,-1.498904,-0.0,0.211314
C40_2023,-9.009689,4.543987,-2.752271,-0.872545,-0.0,0.519387
EX30_2023,-8.956463,4.604384,-1.824008,-0.86739,-0.0,0.441484


In [18]:
elasticity[IV.params.index].mean()

Intercept    -8.746315
Range         3.730757
HP           -1.677749
Chargetime   -1.034015
China        -0.059497
Price         0.547415
dtype: float64

# Cost

In [19]:
fun.cost_firm(logit_data, alpha)
logit_data = fun.markup(logit_data)
logit_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Intercept,China,outside_share,Range_BLP,HP_BLP,Chargetime_BLP,CCP,Model_year,firm_cost,markup%
0,10,2013,0.000006,BMW,I3,29.5,25.00000,16.7,18,Hatchback,...,1.0,0,0.997525,168.5,111.7,237,0.266830,I3_2013,47.040595,-46.854414
1,132,2013,0.001168,Nissan,Leaf,32.8,25.36900,14.7,43,Hatchback,...,1.0,0,0.997525,165.2,113.7,212,0.194320,Leaf_2013,41.420185,-38.752084
2,158,2013,0.000509,Renault,Zoe,36.5,17.31500,13.4,56,Hatchback,...,1.0,0,0.997525,161.5,115.0,199,0.184354,Zoe_2013,32.542945,-46.793383
3,167,2013,0.000006,Smart,Fortwo,12.7,21.04575,8.0,60,Hatchback,...,1.0,0,0.997525,185.3,120.4,195,0.024008,Fortwo_2013,23.028852,-8.611381
4,173,2013,0.000620,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,...,1.0,0,0.997525,137.1,60.9,225,0.213138,Model S_2013,89.561859,-19.657418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,187,2023,0.000388,Volkswagen,up!,25.6,17.48530,8.1,48,Hatchback,...,1.0,0,0.696180,4191.8,2652.3,3227,0.001740,up!_2023,24.256827,-27.915964
330,188,2023,0.002538,Volvo,C40,46.6,43.06649,40.2,28,SUV,...,1.0,0,0.696180,4170.8,2620.2,3247,0.003813,C40_2023,44.474021,-3.164839
331,189,2023,0.000020,Volvo,EX30,47.5,36.82450,26.8,28,SUV,...,1.0,0,0.696180,4169.9,2633.6,3247,0.009698,EX30_2023,38.232031,-3.681550
332,190,2023,0.009647,Volvo,XC40,45.7,43.92666,40.2,28,SUV,...,1.0,0,0.696180,4171.7,2620.2,3247,0.003528,XC40_2023,45.334191,-3.104790


In [20]:
cost_side = sm.ols('np.log(firm_cost) ~ Range + HP + Chargetime', logit_data).fit(cov_type='HC3')
cost_side.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      np.log(firm_cost)   R-squared:                       0.622
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     177.9
Date:                Tue, 04 Jun 2024   Prob (F-statistic):           1.30e-68
Time:                        18:29:38   Log-Likelihood:                -92.539
No. Observations:                 334   AIC:                             193.1
Df Residuals:                     330   BIC:                             208.3
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.1840      0.091     35.128      0.000       3.006       3.362
Range          0.0122      0.003      4.762      0.000       0.007       0.017
HP             0.0165      0.001     10.995      0.000       0.014       0.019
Chargetime    -0.0076      0.001     -5.497      0.000      -0.010      -0.005
==============================================================================
Omnibus:                       31.033   Durbin-Watson:                   1.341
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.296
Skew:                           0.686   Prob(JB):                     1.78e-09
Kurtosis:                       4.007   Cond. No.                         333.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Analysis on subsample

### The subsample consists of the 2023 market with:
The 5 highest market share models: Model Y, Model 3, Enyag iV, ID.4 and Q4 e-tron\
The 5 highest market share chinese models: 4, Euniq6, Atto 3, Marvel R and Dolphin\
The highest ccp model: Ocean\
The highest ccp chinese model: Seal\
The highest ccp korean model (so we have models outside EU, US and china): Ioniq 6\
Polestar 2 as a random interest


In [21]:
analysis_data = logit_data[logit_data['Year']==2023].copy()
analysis_data = analysis_data[analysis_data['Model'].isin(['Model 3', 'Model Y', 'ID.4', 'Enyaq iV', 'Ocean' , '2', 'Ioniq 6', 'Q4 e-tron'
                                                           , '4', 'Euniq6', 'Atto 3', 'Marvel R', 'Dolphin', 'Seal'])]
analysis_data.reset_index(drop=True, inplace=True)
analysis_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Intercept,China,outside_share,Range_BLP,HP_BLP,Chargetime_BLP,CCP,Model_year,firm_cost,markup%
0,5,2023,0.011532,Audi,Q4 e-tron,49.6,65.855307,28.1,28,SUV,...,1.0,0,0.69618,4167.8,2632.3,3247,0.015481,Q4 e-tron_2023,69.250362,-4.902580
1,17,2023,0.001090,BYD,Atto 3,42.0,31.000000,20.1,37,SUV,...,1.0,1,0.69618,4175.4,2640.3,3238,0.003784,Atto 3_2023,32.152076,-3.583209
2,18,2023,0.000997,BYD,Dolphin,42.7,23.323517,9.3,40,Hatchback,...,1.0,1,0.69618,4174.7,2651.1,3235,0.007063,Dolphin_2023,24.475593,-4.707040
3,20,2023,0.000157,BYD,Seal,54.0,32.062000,52.2,38,Sedan,...,1.0,1,0.69618,4163.4,2608.2,3237,0.001324,Seal_2023,33.214076,-3.468638
4,49,2023,0.001424,Fisker,Ocean,70.0,60.237920,56.3,35,SUV,...,1.0,0,0.69618,4147.4,2604.1,3240,0.012322,Ocean_2023,61.255740,-1.661592
5,69,2023,0.000972,Hyundai,Ioniq 6,56.7,43.629166,32.0,16,Sedan,...,1.0,0,0.69618,4160.7,2628.4,3259,0.026387,Ioniq 6_2023,48.810739,-10.615640
6,100,2023,0.001213,Maxus,Euniq6,35.4,37.950676,17.4,45,MPV,...,1.0,1,0.69618,4182.0,2643.0,3230,0.002022,Euniq6_2023,39.158475,-3.084387
7,117,2023,0.003643,MG,4,42.7,26.589191,24.1,41,Hatchback,...,1.0,1,0.69618,4174.7,2636.3,3234,0.002575,4_2023,27.458390,-3.165513
8,119,2023,0.001021,MG,Marvel R,38.8,30.235650,17.7,38,SUV,...,1.0,1,0.69618,4178.6,2642.7,3237,0.003133,Marvel R_2023,31.104849,-2.794416
9,149,2023,0.009927,Polestar,2,51.3,38.989190,41.5,28,Liftback,...,1.0,0,0.69618,4166.1,2618.9,3247,0.005258,2_2023,39.423550,-1.101779


In [22]:
cross_elasticity = fun.cross_elasticity_1(analysis_data, IV)
cross_elasticity

/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:195: PerformanceWarning: indexing past lexsort depth may impact performance.
  cross_elasticity_table.loc[(model_labels[i], model_labels[j], X.columns[k]), 'Cross_Elasticity'] = -coefficients[k] * X.iloc[j, k] * ccp.iloc[j]


Cross_Elasticity
Model_year     Model_year                                
Q4 e-tron_2023 Q4 e-tron_2023 Intercept          0.140015
                              Range             -0.075162
                              HP                 0.029897
                              Chargetime          0.01356
                              China                   0.0
...                                                   ...
ID.4_2023      ID.4_2023      Range             -0.060448
                              HP                 0.023757
                              Chargetime         0.010775
                              China                   0.0
                              Price             -0.006257

[1176 rows x 1 columns]

# Nash Equilibrium on subsample

In [23]:
NE_data = logit_data[logit_data['Year']==2023].copy()
NE_data.reset_index(drop=True, inplace=True)
X_ne = NE_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]

NE_analysis_data = NE_data[NE_data['Model'].isin(['Model 3', 'Model Y', 'ID.4', 'Enyaq iV', 'Ocean' , '2', 'Ioniq 6', 'Q4 e-tron'
                                                           , '4', 'Euniq6', 'Atto 3', 'Marvel R', 'Dolphin', 'Seal'])]
NE_analysis_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,Intercept,China,outside_share,Range_BLP,HP_BLP,Chargetime_BLP,CCP,Model_year,firm_cost,markup%
4,5,2023,0.011532,Audi,Q4 e-tron,49.6,65.855307,28.1,28,SUV,...,1.0,0,0.69618,4167.8,2632.3,3247,0.015481,Q4 e-tron_2023,69.250362,-4.902580
13,17,2023,0.001090,BYD,Atto 3,42.0,31.000000,20.1,37,SUV,...,1.0,1,0.69618,4175.4,2640.3,3238,0.003784,Atto 3_2023,32.152076,-3.583209
14,18,2023,0.000997,BYD,Dolphin,42.7,23.323517,9.3,40,Hatchback,...,1.0,1,0.69618,4174.7,2651.1,3235,0.007063,Dolphin_2023,24.475593,-4.707040
16,20,2023,0.000157,BYD,Seal,54.0,32.062000,52.2,38,Sedan,...,1.0,1,0.69618,4163.4,2608.2,3237,0.001324,Seal_2023,33.214076,-3.468638
26,49,2023,0.001424,Fisker,Ocean,70.0,60.237920,56.3,35,SUV,...,1.0,0,0.69618,4147.4,2604.1,3240,0.012322,Ocean_2023,61.255740,-1.661592
32,69,2023,0.000972,Hyundai,Ioniq 6,56.7,43.629166,32.0,16,Sedan,...,1.0,0,0.69618,4160.7,2628.4,3259,0.026387,Ioniq 6_2023,48.810739,-10.615640
43,100,2023,0.001213,Maxus,Euniq6,35.4,37.950676,17.4,45,MPV,...,1.0,1,0.69618,4182.0,2643.0,3230,0.002022,Euniq6_2023,39.158475,-3.084387
54,117,2023,0.003643,MG,4,42.7,26.589191,24.1,41,Hatchback,...,1.0,1,0.69618,4174.7,2636.3,3234,0.002575,4_2023,27.458390,-3.165513
56,119,2023,0.001021,MG,Marvel R,38.8,30.235650,17.7,38,SUV,...,1.0,1,0.69618,4178.6,2642.7,3237,0.003133,Marvel R_2023,31.104849,-2.794416
74,149,2023,0.009927,Polestar,2,51.3,38.989190,41.5,28,Liftback,...,1.0,0,0.69618,4166.1,2618.9,3247,0.005258,2_2023,39.423550,-1.101779


In [24]:
car1 = 54
car2 = 4

p1_data = NE_data.loc[car1, 'Price'] 
p2_data = NE_data.loc[car2, 'Price'] 

c1 = NE_data.loc[car1, 'firm_cost'] 
c2 = NE_data.loc[car2, 'firm_cost'] 

def market_shares(p1, p2): 
    x2 = NE_data.copy() # copy to avoid writing to the original data 
    fun.set_car_price(x2, p1, car1) # Set new price for car 1
    fun.set_car_price(x2, p2, car2) # set new price for car 2
    ccp = fun.ccp(alpha, beta, x2, X_ne) # compute the CCPs for the new prices
    s1 = ccp[car1] 
    s2 = ccp[car2] 
    return s1, s2

def profit(p, s, c): 
    return s * (p - c)

def profit1(p1, p2): 
    s1, s2 = market_shares(p1, p2)
    pi1 = profit(p1, s1, c1)
    return pi1

def profit2(p2, p1): 
    s1, s2 = market_shares(p1, p2)
    pi2 = profit(p2, s2, c2)
    return pi2

In [25]:
p2 = 0 # initial guess

for i in range(10):
    f = lambda p: -profit1(p1=p, p2=p2)
    res = minimize(f, x0=p2)
    p1 = res['x'][0]
    print(f'BR1(p2={p2:6.4f}) = {p1:6.4f}')

    f = lambda p: -profit2(p1=p1, p2=p)
    res = minimize(f, x0=p2)
    p2 = res['x'][0]
    print(f'BR2(p1={p1:6.4f}) = {p2:6.4f}')

/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encounter

BR1(p2=0.0000) = 1.2890
BR2(p1=1.2890) = 1.2077

/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow e


BR1(p2=1.2077) = 2.5441


/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow e

...

/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow e

BR1(p2=17.2720) = 19.3527


/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow e

BR2(p1=19.3527) = 20.6075


/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow encountered in exp
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: invalid value encountered in true_divide
  ccp_year = np.exp(utility_year) / np.sum(np.exp(utility_year))
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:69: RuntimeWarning: overflow e